# Transformers

In [1]:
from transformers import BertTokenizer, BertModel

#tokenizer = BertTokenizer.from_pretrained("OpenAIGPTLMHeadModel")
model = BertModel.from_pretrained("wietsedv/bert-base-dutch-cased")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


https://huggingface.co/wietsedv/bert-base-dutch-cased?text=gaan+we+%5BMASK%5D+

https://huggingface.co/blog/how-to-generate

In [2]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer


In [4]:
summarizer = pipeline("summarization", model = `'bert-base-multilingual-uncased')
summarizer("Sam Shleifer writes the best docstring examples in the whole world.", min_length=5)


Your max_length is set to 142, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': ' Sam Shleifer writes the best docstring examples in the world .'}]

In [8]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer_NL = AutoTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")
model_NL = AutoModelWithLMHead.from_pretrained("wietsedv/bert-base-dutch-cased")

Some weights of the model checkpoint at wietsedv/bert-base-dutch-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at wietsedv/bert-base-dutch-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
summarizer = pipeline("summarization", model = model_NL, tokenizer=tokenizer_NL)
summarizer("Ik ga nu nmaar huis", min_length=5)


Your max_length is set to 20, but you input_length is only 7. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'Ik ga nu huis naar mijn werk'}]

In [11]:
%%time
tweede_kamer = pd.read_csv("CorpusTweedeKamer.zip")
tweede_kamer = (
    tweede_kamer
    .assign(datum = pd.to_datetime(tweede_kamer.date))
    .assign(speaker = tweede_kamer.speaker.str.lower())
)

CPU times: user 12.7 s, sys: 919 ms, total: 13.6 s
Wall time: 13.8 s


In [54]:
sample_texts = tweede_kamer.query("party == 'VVD'").sample(1).text.values.tolist()

In [55]:
inputs = " ".join(sample_texts)

In [56]:
inputs

'De argumenten zijn al gegeven in de schriftelijke inbreng; daar staan zij aangegeven. Wij hebben allen, zo mag ik aannemen, veel reacties gekregen en gesprekken gevoerd met vakbeweging, met werkgevers, met organisaties op dit terrein. Wij kennen de argumenten. U vraagt almaar door naar de bekende weg. Nogmaals: ik ga het niet uitlokken, ik wacht niet tot het gebeurd is. Dan zitten wij namelijk hier bij elkaar en zeggen wij: tjonge, wat erg dat dit toch gebeurd is.'

In [57]:
input_ids = tokenizer.encode(inputs, return_tensors='pt')

In [58]:
input_ids.shape

torch.Size([1, 99])

In [59]:
N = input_ids.shape[1]

In [66]:
greedy_output = model.generate(input_ids, max_length= N +20)

In [67]:
tt = len(inputs)

In [68]:
print(inputs)
print("--------------")
out = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
zz = len(out)
out[tt:zz]


De argumenten zijn al gegeven in de schriftelijke inbreng; daar staan zij aangegeven. Wij hebben allen, zo mag ik aannemen, veel reacties gekregen en gesprekken gevoerd met vakbeweging, met werkgevers, met organisaties op dit terrein. Wij kennen de argumenten. U vraagt almaar door naar de bekende weg. Nogmaals: ik ga het niet uitlokken, ik wacht niet tot het gebeurd is. Dan zitten wij namelijk hier bij elkaar en zeggen wij: tjonge, wat erg dat dit toch gebeurd is.
--------------


'dat is niet zo erg, hoor. Nogmaals : wij vragen het niet meer meer terug'

## summarizer

In [37]:
tokenizer_NL = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased')
model_NL = AutoModelWithLMHead.from_pretrained('bert-base-multilingual-uncased')

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
summarizer = pipeline("summarization", model = model_NL, tokenizer= tokenizer_NL)

In [13]:
tweede_kamer

,date,agenda,speechnumber,speaker,party,party.facts.id,chair,terms,text,parliament,iso3country,datum
0,1994-12-20,NaN,1,marijnissen,SP,1363.0,False,561,Mijnheer de voorzitter! Ik vertel de minister ...,NL-TweedeKamer,NLD,1994-12-20
1,1994-12-20,NaN,2,melkert,PvdA,1234.0,False,706,Mijnheer de voorzitter! Mag ik allereerst de h...,NL-TweedeKamer,NLD,1994-12-20
2,1994-12-20,NaN,3,marijnissen,SP,1363.0,False,304,Mijnheer de voorzitter! Hoewel ik het antwoord...,NL-TweedeKamer,NLD,1994-12-20
3,1994-12-20,NaN,4,melkert,PvdA,1234.0,False,374,Mijnheer de voorzitter! Wat is onrechtvaardig?...,NL-TweedeKamer,NLD,1994-12-20
4,1994-12-20,NaN,5,rosenmöller,GL,1537.0,False,412,Voorzitter! Afgelopen zaterdag stond in NRC Ha...,NL-TweedeKamer,NLD,1994-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...
1143361,2019-07-04,NaN,965,staatssecretaris broekers-knol,NaN,NaN,False,142,Dan de motie op stuk nr. 2519 van de heer Hidd...,NL-TweedeKamer,NLD,2019-07-04
1143362,2019-07-04,NaN,966,de voorzitter,NaN,NaN,True,3,Dank u wel.,NL-TweedeKamer,NLD,2019-07-04
1143363,2019-07-04,NaN,967,de voorzitter,NaN,NaN,True,62,Over exact 60 minuten gaan wij stemmen over de...,NL-TweedeKamer,NLD,2019-07-04
1143364,2019-07-04,NaN,968,mevrouw van toorenburg,CDA,1157.0,False,21,Toch nog even — misschien is het allemaal afge...,NL-TweedeKamer,NLD,2019-07-04


In [39]:
lange_tekst = (
    tweede_kamer
    .query("terms > 100")
    .query("terms < 512")
    .query("party == 'VVD'")
    .sample(1)
    .text.values.tolist()
)

In [40]:
lange_tekst[0]

'Gisteren heb ik ook heel duidelijk gezegd dat altijd het "h-woord" genoemd wordt in geval van hervormingen. Ik heb ook aangegeven wat het vorige kabinet daaraan gedaan heeft en dat het nu tijd wordt om eveneens iets aan het huurstelsel te doen. Gelukkig hoorde ik de minister zeggen dat daar ook naar gekeken wordt. De minister kan wel aangeven dat het niet rechtvaardig is, meer ondersteund te worden of om meer hypotheekrenteaftrek te krijgen als je meer verdient. Ik wijs er echter wel op dat mensen die meer verdienen, meer belasting betalen. Wij zullen wat dat betreft een evenwicht moeten vinden. Ik meen dat het huidige kabinet ook het eigen woningforfait voor de echt dure huizen al heeft verhoogd. Volgens mij is het bedrag echter niet geïndexeerd. Uiteindelijk zijn wij wat dat betreft dus ook allemaal aan de beurt.'

In [41]:
summarizer(lange_tekst[0], min_length=10, max_length = 300)


AssertionError: The context has 208 number of tokens, but `max_length` is only 200. Please make sure that `max_length` is bigger than the number of tokens, by setting either `generate(max_length=...,...)` or `config.max_length = ...`